# Llama2-7B-LoRA-8bit

LLM.int8()：混合精度分解量化，加载模型时开启load_in_8bit=True

训练和推理速度都降低，一般不用它训练，而用QLoRA

加载显存7.284G，训练显存12.016G（batchsize为1且不开梯度检查点时），开model.half则adam_epsilon要调大

## Step1 导入相关包

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "4"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [ ]:
ds = load_dataset("lifefabric/alpaca_data_cleaned.zh", split="train[:10000]")
ds

In [ ]:
ds[:3]

## Step3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/node6_1/tanshuai/modelscope/Llama-2-7b-ms")
tokenizer

In [ ]:
tokenizer.pad_token_id = 2

In [ ]:
def process_func(example):
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
print(tokenized_ds[0]["input_ids"])

In [ ]:
# tokenizer.decode(tokenized_ds[0]["input_ids"])

In [ ]:
tokenizer(tokenizer.eos_token)

In [ ]:
# tokenizer('abc' + tokenizer.eos_token)

In [ ]:
# Llama的词表没针对中文设计，一个字解码出多个tokens
tokenizer("呀", add_special_tokens=False)

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## Step4 创建模型

In [ ]:
import torch
# 多卡情况，可以去掉device_map="auto"，否则会将模型拆开
model = AutoModelForCausalLM.from_pretrained("/node6_1/tanshuai/modelscope/Llama-2-7b-ms", 
                                             low_cpu_mem_usage=True, 
                                             torch_dtype=torch.half, 
                                             device_map="auto",
                                             load_in_8bit=True)

In [ ]:
model.dtype

In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

In [ ]:
model.config

## LoRA

### PEFT 1 配置文件

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(task_type=TaskType.CAUSAL_LM)
print(config)

### PEFT 2 创建模型

In [ ]:
model = get_peft_model(model, config)

In [ ]:
print(config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

In [ ]:
model.half()

In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

## Step5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    # gradient_checkpointing=True,
    adam_epsilon=1e-4    # 当整个模型都是半精度时，需要将adam_epsilon调大
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds.select(range(5000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## Step7 模型训练

In [ ]:
trainer.train()

## Step8 模型推理

In [ ]:
# 生成时间明显变长
model.eval()
ipt = tokenizer("Human: {}\n{}".format("考试有什么技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

In [ ]:
# 最好不要合并，由于量化过程中的四舍五入误差，可能会导致生成的结果与不合并时有所不同。
model.merge_and_unload()

In [ ]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)